In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import time
import yaml
import datetime
import pickle
from multiprocessing import Queue

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

import sys
sys.path.append('..')
from config import CFG
CFG=CFG()

import warnings; warnings.simplefilter('ignore')

In [8]:
class Crawler():
    def __init__(self, *chrome_options, url=CFG.URL2, window_size=CFG.WINDOW_SIZE):
        # sourcery skip: remove-pass-body
        self.url = url
        self.window_size = window_size
        if chrome_options:
            self.options = webdriver.ChromeOptions()
            for option in chrome_options:
                self.options.add_argument(option)
            opt = True
        else: opt = False
        self.opt = opt

    def connect(self):
        print("Connecting to Selenium")
        if self.opt:
            self.driver = webdriver.Chrome(options=self.options)
        else:
            self.driver = webdriver.Chrome()
        self.driver.get(self.url)
        self.driver.set_window_size(self.window_size[0], self.window_size[1])
        time.sleep(40)
        self.driver.switch_to.frame(0)
        self.driver.find_element(By.CSS_SELECTOR, "#tab_container > li:nth-child(3) > a").click() # '//*[@id="tab_container"]/li[3]/a'
        self.pages = self.driver.find_element(By.CLASS_NAME,
            'last-page'
        ).text  #table-ogu706xi71r > div.previous-next-container > div > div.last-page  //*[@id="table-ogu706xi71r"]/div[3]/div/div[2]
        print("Successfully connected to Selenium")

    def __del__(self):
        self.driver.quit()

    def click_button(self, selector, value, sleep_time=0):
        button = self.driver.find_element(selector, value)
        self.driver.execute_script("arguments[0].click();", button)
        time.sleep(sleep_time)

    def make_df(self):
        page_source = self.driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        table = soup.find('table')
        return pd.read_html(str(table))[0]

    def collect_data(self, num_clicks=None, start_page=0, save=True):
        previous_date = None
        print("Starting data collection")
        if num_clicks is None:
            num_clicks = int(self.pages) - 1
        if start_page != 0:
            self.driver.find_element(By.CLASS_NAME, "current-page").click()
            self.driver.find_element(By.CLASS_NAME, "current-page").send_keys(str(start_page))
            self.driver.find_element(By.CLASS_NAME, "current-page").send_keys(Keys.ENTER)
        dfs = [self.make_df()]
        for i in range(num_clicks):
            WebDriverWait(self.driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "next-page"))
            )
            self.click_button(By.CLASS_NAME, "next-page", 0.1)
            dfs.append(self.make_df())
            if i % 100 == 0:
                print(f"Collected {i} pages")
            if i % 500 == 0:
                if previous_date is not None:
                    try:
                        os.remove(f'data_files/df_lists-{previous_date}.pkl')
                    except Exception as e:
                        print(f'Unable to delete previous saved file because of: {e}')
                if save:
                    print("Saving data")
                    previous_date = datetime.date.today()
                    pickle.dump(dfs, open(f'data_files/df_lists-{previous_date}.pkl', 'wb'))
        print("Finished data collection")
        return pd.concat(dfs, ignore_index=True).reset_index(drop=True)

    def run(self, num_clicks=None, num_threads=1, save=True):
        # sourcery skip: remove-pass-body, remove-redundant-if
        if num_threads > 1:
            pass
        else:
            df = self.collect_data(num_clicks, save=save)
        self.teardown_method()
        df.columns = df.columns.str.replace('  ', ' ')
        df = df.dropna(how='all')
        if save:
            today = datetime.date.today()
            df.to_csv(f'/data_files/scraped_data-{today}.csv', index=False)
        return df

In [9]:
crawler = Crawler()#(*CFG.CHROME_OPTIONS)

In [10]:
crawler.connect()

Connecting to Selenium
Successfully connected to Selenium


In [6]:
crawler.run(num_clicks=5, save=False)

Starting data collection


TimeoutException: Message: 


In [11]:
crawler.pages

'3126'

In [ ]:
from multiprocessing import Queue

In [ ]:
q = Queue()

In [ ]:
q.put_nowait()